# Get IPs

In this notebook we explore how to get IPs from our IP list.

In [1]:
import requests

In [2]:
url = "https://freegeoip.app/csv/"

headers = {
    'accept': "application/csv",
    'content-type': "application/csv"
    }


response = requests.request("GET", url, headers=headers)

print(response.text)

24.17.221.217,US,United States,WA,Washington,Liberty Lake,99019,America/Los_Angeles,47.6517,-117.0838,881



In [3]:
url_ip = "https://freegeoip.app/csv/{}"

In [4]:
response = requests.request("GET", url_ip.format('23.116.185.114'))

In [5]:
print(response.text)

23.116.185.114,US,United States,IL,Illinois,Wheaton,60187,America/Chicago,41.8733,-88.1163,602



In [8]:
response.status_code

200

In [9]:
response2 = requests.request("GET", url_ip.format('::ffff:185.40.4.65'))

In [10]:
response2.text

'185.40.4.65,RU,Russia,,,,,Europe/Moscow,55.7386,37.6068,0\r\n'

In [11]:
response2.status_code

200